In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
np.random.seed(42)

In [84]:
total_clicks=pd.read_csv('raw_data/train.csv')

In [3]:
clicks=pd.read_csv('raw_data/mi_clicks.csv')
print(clicks.columns)

Index(['id', 'click', 'hour', 'C1', 'banner_pos', 'site_id', 'site_domain',
       'site_category', 'app_id', 'app_domain', 'app_category', 'device_id',
       'device_ip', 'device_model', 'device_type', 'device_conn_type', 'C14',
       'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'time', 'day'],
      dtype='object')


In [6]:
clicks.nunique().sort_values()

click                     2
C18                       4
device_conn_type          4
device_type               4
C1                        7
banner_pos                7
C15                       8
C16                       9
day                      10
site_category            21
hour                     24
app_category             26
C21                      60
C19                      66
C20                     163
app_domain              180
time                    240
C17                     421
C14                    2208
site_id                2648
site_domain            2887
app_id                 3074
device_model           5193
device_id            145435
device_ip            558611
id                  1000000
dtype: int64

In [27]:
sample_clicks=clicks.loc[np.random.choice(len(clicks),int(len(clicks)*0.1),replace=True)].sort_values('time')
sample_clicks.index=np.arange(len(sample_clicks))
cols=clicks.columns
not_need_cols=['id','click']
cols_unique=sorted([(c,len(clicks[c].unique())) for c in set(cols)-set(not_need_cols)],key=lambda x:x[1], reverse=True)
sample_cols_unique=sorted([(c,len(sample_clicks[c].unique())) for c in set(cols)-set(not_need_cols)],key=lambda x:x[1], reverse=True)
sample_cols_ratio=sorted([(tc[0],tc[1],sc[1],sc[1]/tc[1]) for tc in cols_unique for sc in sample_cols_unique if tc[0] == sc[0] ],key=lambda x:x[3])
sample_cols_ratio

[('device_id', 145435, 15271, 0.1050022346752845),
 ('device_ip', 558611, 74190, 0.1328115629659996),
 ('app_id', 3074, 1239, 0.4030579050097593),
 ('site_domain', 2887, 1290, 0.44683062002078283),
 ('app_domain', 180, 85, 0.4722222222222222),
 ('site_id', 2648, 1419, 0.5358761329305136),
 ('device_model', 5193, 3103, 0.5975351434623531),
 ('C14', 2208, 1674, 0.7581521739130435),
 ('app_category', 26, 21, 0.8076923076923077),
 ('C20', 163, 152, 0.9325153374233128),
 ('C17', 421, 395, 0.9382422802850356),
 ('site_category', 21, 20, 0.9523809523809523),
 ('C19', 66, 64, 0.9696969696969697),
 ('time', 240, 240, 1.0),
 ('C21', 60, 60, 1.0),
 ('hour', 24, 24, 1.0),
 ('day', 10, 10, 1.0),
 ('C16', 9, 9, 1.0),
 ('C15', 8, 8, 1.0),
 ('banner_pos', 7, 7, 1.0),
 ('C1', 7, 7, 1.0),
 ('device_conn_type', 4, 4, 1.0),
 ('C18', 4, 4, 1.0),
 ('device_type', 4, 4, 1.0)]

In [86]:

def split_train_test(clicks):
    tra_end_idx=len(clicks[clicks.hour<14300000])

    drop_cols=['id','click','hour','device_id','device_ip',]
#     [c for c in clicks.columns if c.startswith('site') or c.startswith('device')]
#     cate_cols=set(clicks.columns)-set(drop_cols)
#     print(cate_cols)
#     clicks=pd.get_dummies(clicks, columns=cate_cols)

    t_cate_cols=[c for c in clicks.columns if clicks[c].dtype==np.dtype('O')]
    for c in clicks.columns:
        le=LabelEncoder()
        clicks[c]=le.fit_transform(clicks[c])

    train_clicks=clicks[:tra_end_idx]
    test_clicks=clicks[tra_end_idx:]

    X_train=train_clicks.drop(drop_cols,axis=1)
    y_train=train_clicks['click']
    X_test=test_clicks.drop(drop_cols,axis=1)
    y_test=test_clicks['click']
    
    return X_train,X_test,y_train,y_test

X_train,X_test,y_train,y_test = split_train_test(total_clicks)

len(X_train.columns)

19

In [ ]:
X_train.shape,X_test.shape

In [87]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression,Lasso,LogisticRegressionCV,RidgeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss,roc_auc_score
from lightgbm import LGBMClassifier

classifies = [
#     RidgeClassifier(),
#     LogisticRegression(verbose=1,random_state=42),
#     Lasso(random_state=42),
#               RandomForestClassifier(verbose=1,random_state=42,n_jobs=4,max_depth=50,n_estimators=500),
#               GradientBoostingClassifier(verbose=1,random_state=42),
    LGBMClassifier(random_state=42,verbose=1),
             ]

scores=[]
for c in classifies:
    
    if c is LGBMClassifier:
        c.fit(X_train,y_train,categorical_feature=list(X_train.columns))
    else:
        c.fit(X_train,y_train)
    scores.append((c,roc_auc_score(y_test,c.predict(X_test))))
    
sorted(scores,key=lambda x:x[1],reverse=True)


ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.

In [ ]:
gbc=GradientBoostingClassifier()
gbc.fit(sample_clicks)

In [ ]:
classifies[0].coef_.shape

In [ ]:
sorted([(X_train.columns[i],classifies[0].coef_[0][i]) for i in range(len(X_train.columns))],key=lambda x:x[1],reverse=True)